In [5]:
using JuMP
using MathProgBase.SolverInterface
using GLPKMathProgInterface
using LazySets
include("../src/utils/activation.jl")
include("../src/utils/network.jl")
include("../src/utils/problem.jl")
include("../src/utils/util.jl")
include("../src/feasibility/utils/feasibility.jl")
include("../src/feasibility/iLP.jl")
include("../src/reachability/maxSens.jl")

partition (generic function with 1 method)

In [34]:
import JuMP: GenericAffExpr

In [12]:
small_nnet = read_nnet("../examples/networks/small_nnet.txt")

Network(Layer[Layer([1.0; 1.0], [1.5, 1.5], ReLU()), Layer([2.0 2.0; 2.0 2.0], [2.5, 2.5], ReLU()), Layer([3.0 3.0], [3.5], ReLU())])

In [22]:
small_nnet

Network(Layer[Layer([1.0; 1.0], [1.5, 1.5], ReLU()), Layer([2.0 2.0; 2.0 2.0], [2.5, 2.5], ReLU()), Layer([3.0 3.0], [3.5], ReLU())])

In [407]:
A = Array{Float64, 2}(2,1)
A[1,1] = 1
A[2,1] = -1
inputSet = HPolytope(A, [1.0,1.0])
outputSet = HPolytope(A[1:1, :],[50.0])

LazySets.HPolytope{Float64}(LazySets.HalfSpace{Float64}[LazySets.HalfSpace{Float64}([1.0], 50.0)])

In [408]:

problem = Problem(small_nnet, inputSet, outputSet)
optimizer = GLPKSolverMIP()
#solver = Reverify(optimizer, 1000.0)
#solve(solver, problem)

GLPKMathProgInterface.GLPKInterfaceMIP.GLPKSolverMIP(false, Any[])

# Parse the Network

In [ ]:
small_nnet = read_nnet("../examples/networks/small_nnet.txt")

# Setup the equations and keep track of ReLUs

In [25]:
network = small_nnet

Network(Layer[Layer([1.0; 1.0], [1.5, 1.5], ReLU()), Layer([2.0 2.0; 2.0 2.0], [2.5, 2.5], ReLU()), Layer([3.0 3.0], [3.5], ReLU())])

In [409]:
#model = JuMP.Model(solver = solver.optimizer)
model = JuMP.Model(solver = GLPKSolverMIP())

Feasibility problem with:
 * 0 linear constraints
 * 0 variables
Solver is GLPKInterfaceMIP

In [40]:
layers = network.layers
neurons = Vector{Vector{Variable}}(length(layers) + 1) # +1 for input layer
input_layer_n = size(first(layers).weights, 2)
all_layers_n  = [length(l.bias) for l in layers]
insert!(all_layers_n, 1, input_layer_n)

#todo encode relus

for (i, n) in enumerate(all_layers_n)
    neurons[i] = @variable(model, [1:n]) # To do: name the variables
    deltas[i]  = @variable(model, [1:n], Bin)
end

In [36]:
model = JuMP.Model(solver = GLPKSolverLP(method=:Exact))

Feasibility problem with:
 * 0 linear constraints
 * 0 variables
Solver is GLPKInterfaceLP

In [ ]:
 neurons, deltas = init_nnet_vars(solver, model, problem.network)

In [32]:
all_layers_n

3-element Array{Int64,1}:
 2
 2
 1

In [45]:
model

Feasibility problem with:
 * 0 linear constraints
 * 12 variables: 6 binary
Solver is GLPKInterfaceMIP

In [29]:
deltas

4-element Array{Array{JuMP.Variable,1},1}:
 #undef
 #undef
 #undef
 #undef

#### activation assignment search goes here later

# Solve the associated LP

In [3]:
m = Model(solver = GLPKSolverLP(method=:Exact))

@variable(m, x1 >= 0)
@variable(m, x2 >= 0)

@objective(m, Max, 15*x1 + 10*x2)

@constraint(m, .25*x1 + x2 <= 65) # Adds the constraint x + y <= 1
@constraint(m, 1.25*x1 + .5*x2 <= 90) # Adds the constraint x + y <= 1

m

Maximization problem with:
 * 2 linear constraints
 * 2 variables
Solver is GLPKInterfaceLP

In [7]:
aux = [x1, x2]

2-element Array{JuMP.Variable,1}:
 x1
 x2

In [9]:
getvalue(aux[1])

51.11111111111111

In [4]:
solve(m)

:Optimal

In [6]:
getobjectivevalue(m)

1288.8888888888887

In [ ]:
include("../src/feasibility/utils/feasibility.jl")
#include("../src/feasibility/iLP.jl")
small_nnet = read_nnet("../examples/networks/small_nnet.txt")
A = Array{Float64, 2}(2,1)
A[1,1] = 1
A[2,1] = -1
inputSet = Hyperrectangle([-20.0], [.5])
outputSet = HPolytope(A[1:1, :],[102.5])
problem = Problem(small_nnet, inputSet, outputSet)
optimizer = GLPKSolverMIP()
solver = ILP(optimizer, 1)
solve(solver, problem)

In [32]:
small_nnet = read_nnet("../examples/networks/small_nnet.txt")
A = Array{Float64, 2}(2,1)
A[1,1] = 1
A[2,1] = -1
inputSet = Hyperrectangle([-20.0], [.5])
inputSet = Hyperrectangle([-20.0], [0.0])
outputSet = HPolytope(A[1:1, :],[102.5])
problem = Problem(small_nnet, inputSet, outputSet)
solver = GLPKSolverLP(method=:Exact)
m = Model(solver=solver)

Feasibility problem with:
 * 0 linear constraints
 * 0 variables
Solver is GLPKInterfaceLP

In [16]:
small_nnet

Network(Layer[Layer([1.0; 1.0], [1.5, 1.5], ReLU()), Layer([2.0 2.0; 2.0 2.0], [2.5, 2.5], ReLU()), Layer([3.0 3.0], [3.5], ReLU())])

In [10]:
inputSet.radius

1-element Array{Float64,1}:
 0.5

In [19]:
bounds = get_bounds(problem)

4-element Array{LazySets.Hyperrectangle,1}:
 LazySets.Hyperrectangle{Float64}([-20.0], [0.0])        
 LazySets.Hyperrectangle{Float64}([0.0, 0.0], [0.0, 0.0])
 LazySets.Hyperrectangle{Float64}([2.5, 2.5], [0.0, 0.0])
 LazySets.Hyperrectangle{Float64}([18.5], [0.0])         

In [13]:
Hyperrectangle(low=[0.0, 0.0, 0.0], high=[1.0,2.0,3.0])

LazySets.Hyperrectangle{Float64}([0.5, 1.0, 1.5], [0.5, 1.0, 1.5])

# Clean functions

In [ ]:
function encode(solver::ILP, model::Model, problem::Problem, act_pattern::Vector{Vector{Bool}})
    neurons, deltas = init_nnet_vars(solver, model, problem.network)
    add_complementary_output_constraint(model, problem.output, last(neurons))

    for (i, layer) in enumerate(problem.network.layers)
        (W, b, act) = (layer.weights, layer.bias, layer.activation)
        before_act = W * neurons[i] + b
        for j in 1:length(layer.bias) # For evey node
            if act_pattern[i][j]
                # @constraint(model, before_act[j] >= 0.0)
                @constraint(model, neurons[i+1][j] == before_act[j])
            else
                # @constraint(model, before_act[j] <= 0.0)
                @constraint(model, neurons[i+1][j] == 0.0)
            end
        end
    end

    # Objective: L∞ norm of the disturbance
    J = maximum(symbolic_abs(neurons[1] - problem.input.center))
    @objective(model, Min, J)

    return neurons[1]
end

In [29]:
solver = GLPKSolverLP(method=:Exact)

GLPKMathProgInterface.GLPKInterfaceLP.GLPKSolverLP(false, :Exact, Any[])

## attempt to programatically name variables

### need to investigate how to do this

In [251]:
function string_as_varname(s::AbstractString,v::Any)
         s=symbol(s)
         @eval (($s) = ($v))
end

string_as_varname (generic function with 1 method)

In [253]:
test

2-element Array{JuMP.Variable,1}:
 x1
 x2

In [259]:
testbs = [[@variable(m, b11)],
         [@variable(m, b21),@variable(m, b22)],
         [@variable(m, b31),@variable(m, b32)],  
         [@variable(m, b41)]]

testfs = [[@variable(m, f21),@variable(m, f22)],
          [@variable(m, f31),@variable(m, f32)]]

4-element Array{Array{JuMP.Variable,1},1}:
 JuMP.Variable[b11]     
 JuMP.Variable[b21, b22]
 JuMP.Variable[b31, b32]
 JuMP.Variable[b41]     

In [255]:
test[1]

x1

In [254]:
string_as_varname("test", test[1])

LoadError: [91mUndefVarError: symbol not defined[39m

In [250]:
test = [@variable(m, "x1"), @variable(m, x2)]

LoadError: [91mIn @variable(m,x1): Expected x1 to be a variable name[39m

In [249]:
test

2-element Array{JuMP.Variable,1}:
 x1
 x2

In [240]:
function init_nnet_vars(model::Model, network::Network)
    layers = network.layers
    #input layer and last layer have b_vars because they are unbounded
    b_vars = Vector{Vector{Variable}}(length(layers) + 1) # +1 for input layer
    #f_vars are always positive and used as front for ReLUs
    f_vars = Vector{Vector{Variable}}(length(layers) -1)
    
    input_layer_n = size(first(layers).weights, 2)
    all_layers_n  = [length(l.bias) for l in layers]
    insert!(all_layers_n, 1, input_layer_n)

    for (i, n) in enumerate(all_layers_n)
        b_vars[i] = @variable(model, [1:n]) # To do: name the variables
        if 1 < i < length(layers) + 1
            f_vars[i-1] = @variable(model, [1:n])
        end
    end
    return b_vars, f_vars
end

init_nnet_vars (generic function with 2 methods)

In [241]:
bs, fs = init_nnet_vars(m, problem.network)

(Array{JuMP.Variable,1}[__anon__[i] ∀ i ∈ {1}, __anon__[i] ∀ i ∈ {1,2}, __anon__[i] ∀ i ∈ {1,2}, __anon__[i] ∀ i ∈ {1}], Array{JuMP.Variable,1}[__anon__[i] ∀ i ∈ {1,2}, __anon__[i] ∀ i ∈ {1,2}])

In [246]:
test = @variable(m)

__anon__

In [247]:
test.name

Maximization problem with:
 * 14 linear constraints
 * 31 variables
Solver is GLPKInterfaceLP

In [245]:
for layer in bs
    for var in layer
        var.set("as")
    end
end

LoadError: [91mtype Variable has no field setName[39m

In [136]:
fs

2-element Array{Array{JuMP.Variable,1},1}:
 __anon__[i] ∀ i ∈ {1,2}
 __anon__[i] ∀ i ∈ {1,2}

In [137]:
relu_status = [[1,1],[1,1]]

2-element Array{Array{Int64,1},1}:
 [1, 1]
 [1, 1]

In [401]:
function add_input_constraint(model::Model, input::HPolytope, neuron_vars::Vector{Variable})
    in_A,  in_b  = tosimplehrep(input)
    @constraint(model,  in_A * neuron_vars .<= in_b)
    return nothing
end

function add_input_constraint(model::Model, input::Hyperrectangle, neuron_vars::Vector{Variable})
    @constraint(model,  neuron_vars .<= high(input))
    @constraint(model,  neuron_vars .>= low(input))
    return nothing
end

function add_output_constraint(model::Model, output::AbstractPolytope, neuron_vars::Vector{Variable})
    out_A, out_b = tosimplehrep(output)
    @constraint(model, out_A * neuron_vars .<= out_b)
    return nothing
end

add_output_constraint (generic function with 1 method)

In [151]:
get_bounds(problem)

4-element Array{LazySets.Hyperrectangle,1}:
 LazySets.Hyperrectangle{Float64}([-20.0], [0.0])        
 LazySets.Hyperrectangle{Float64}([0.0, 0.0], [0.0, 0.0])
 LazySets.Hyperrectangle{Float64}([2.5, 2.5], [0.0, 0.0])
 LazySets.Hyperrectangle{Float64}([18.5], [0.0])         

In [154]:
bs[2][2]

__anon__[2]

In [402]:
function encode(model::Model, problem::Problem, relustatus::Array{Array{Int64,1},1})
    #bs, fs = init_nnet_vars(model, problem.network)
    
    #TEST
    bs = [[@variable(m, b11)],
         [@variable(m, b21),@variable(m, b22)],
         [@variable(m, b31),@variable(m, b32)],  
         [@variable(m, b41)]]

    fs = [[@variable(m, f21),@variable(m, f22)],
          [@variable(m, f31),@variable(m, f32)]]
    
    add_input_constraint(model, inputSet, bs[1])
    #add_complementary_output_constraint(model, problem.output, last(neurons))

    for (i, layer) in enumerate(problem.network.layers)
        print(string("i: ",i))
        (W, b, act) = (layer.weights, layer.bias, layer.activation)
        #before_act = W * neurons[i] + b
        
        #first layer is different
        if i == 1
            for j in 1:length(layer.bias)
               @constraint(model, -bs[2][j] + bs[1][1]*W[j] == -b[j]) 
            end
        elseif 1<i
            for j in 1:length(layer.bias) # For evey node
                @constraint(model, -bs[i+1][j] + dot(fs[i-1],W[j,:]) == -b[j])
            end
        end
    end

    # Objective: L∞ norm of the disturbance\
    @objective(m, Max, 0)
end

encode (generic function with 2 methods)

In [ ]:
#modify assignment to 

# TODO:

 - debug relu_status
 - debug fix_relus
 - debug SMT decision
 - investigate copy() for JuMP models 
 - refactor into correct design
 - figure out variable naming (JuMP documentation has been offline almost all weekend)
 - 
 

In [410]:
m = Model(solver = GLPKSolverLP(method=:Exact))

Feasibility problem with:
 * 0 linear constraints
 * 0 variables
Solver is GLPKInterfaceLP

In [404]:
encode(m, problem, [[1,1], [1,1]])

i: 1i: 2i: 3

0

In [405]:
m

Maximization problem with:
 * 7 linear constraints
 * 10 variables
Solver is GLPKInterfaceLP

In [406]:
get_bounds(problem)

4-element Array{LazySets.Hyperrectangle,1}:
 LazySets.Hyperrectangle{Float64}([-20.0], [0.0])        
 LazySets.Hyperrectangle{Float64}([0.0, 0.0], [0.0, 0.0])
 LazySets.Hyperrectangle{Float64}([2.5, 2.5], [0.0, 0.0])
 LazySets.Hyperrectangle{Float64}([18.5], [0.0])         

In [392]:
get_bounds(problem)

4-element Array{LazySets.Hyperrectangle,1}:
 LazySets.Hyperrectangle{Float64}([-20.0], [0.0])        
 LazySets.Hyperrectangle{Float64}([0.0, 0.0], [0.0, 0.0])
 LazySets.Hyperrectangle{Float64}([2.5, 2.5], [0.0, 0.0])
 LazySets.Hyperrectangle{Float64}([18.5], [0.0])         

In [393]:
solve(m)

LoadError: [91mMethodError: no method matching solve(::JuMP.Model)[0m
Closest candidates are:
  solve([91m::ILP[39m, [91m::Problem[39m) at /Users/christopherlazarus/Documents/Stanford/Research/clean/NeuralVerification.jl/src/feasibility/iLP.jl:9
  solve([91m::Feasibility[39m, [91m::Problem[39m) at /Users/christopherlazarus/Documents/Stanford/Research/clean/NeuralVerification.jl/src/feasibility/utils/feasibility.jl:7
  solve([91m::MaxSens[39m, [91m::Problem[39m) at /Users/christopherlazarus/Documents/Stanford/Research/clean/NeuralVerification.jl/src/reachability/maxSens.jl:10[39m

In [214]:
bs

4-element Array{Array{JuMP.Variable,1},1}:
 __anon__[i] ∀ i ∈ {1}  
 __anon__[i] ∀ i ∈ {1,2}
 __anon__[i] ∀ i ∈ {1,2}
 __anon__[i] ∀ i ∈ {1}  

In [194]:
fs

2-element Array{Array{JuMP.Variable,1},1}:
 __anon__[i] ∀ i ∈ {1,2}
 __anon__[i] ∀ i ∈ {1,2}

In [212]:
fs[1]

2-element Array{JuMP.Variable,1}:
 __anon__[1]
 __anon__[2]

In [211]:
problem.network.layers[2].weights[1,:]

2-element Array{Float64,1}:
 2.0
 2.0

In [205]:
problem.network.layers[1].weights

2×1 Array{Float64,2}:
 1.0
 1.0

In [184]:
bs

4-element Array{Array{JuMP.Variable,1},1}:
 __anon__[i] ∀ i ∈ {1}  
 __anon__[i] ∀ i ∈ {1,2}
 __anon__[i] ∀ i ∈ {1,2}
 __anon__[i] ∀ i ∈ {1}  

In [178]:
bs[3]

2-element Array{JuMP.Variable,1}:
 __anon__[1]
 __anon__[2]

In [183]:
for (i, layer) in enumerate(problem.network.layers)
    print(i)
end

123

In [179]:
fs

2-element Array{Array{JuMP.Variable,1},1}:
 __anon__[i] ∀ i ∈ {1,2}
 __anon__[i] ∀ i ∈ {1,2}

In [169]:
problem.network.layers[1].weights

2×1 Array{Float64,2}:
 1.0
 1.0

In [176]:
bs

4-element Array{Array{JuMP.Variable,1},1}:
 __anon__[i] ∀ i ∈ {1}  
 __anon__[i] ∀ i ∈ {1,2}
 __anon__[i] ∀ i ∈ {1,2}
 __anon__[i] ∀ i ∈ {1}  

In [ ]:
fs[]

In [172]:
bs[1]

1-element Array{JuMP.Variable,1}:
 __anon__[1]

In [164]:
fs

2-element Array{Array{JuMP.Variable,1},1}:
 __anon__[i] ∀ i ∈ {1,2}
 __anon__[i] ∀ i ∈ {1,2}

In [165]:
bs

4-element Array{Array{JuMP.Variable,1},1}:
 __anon__[i] ∀ i ∈ {1}  
 __anon__[i] ∀ i ∈ {1,2}
 __anon__[i] ∀ i ∈ {1,2}
 __anon__[i] ∀ i ∈ {1}  

In [181]:
2:length(problem.network.layers[1].bias)

2:2

In [160]:
problem.network.layers[1].bias

2-element Array{Float64,1}:
 1.5
 1.5

In [131]:
bs, fs = init_nnet_vars(m, problem.network)

(Array{JuMP.Variable,1}[__anon__[i] ∀ i ∈ {1}, __anon__[i] ∀ i ∈ {1,2}, __anon__[i] ∀ i ∈ {1,2}, __anon__[i] ∀ i ∈ {1}], Array{JuMP.Variable,1}[__anon__[i] ∀ i ∈ {1,2}, __anon__[i] ∀ i ∈ {1,2}])

In [96]:
bs[1]

1-element Array{JuMP.Variable,1}:
 __anon__[1]

In [62]:
fs

4-element Array{Array{JuMP.Variable,1},1}:
 __anon__[i] ∀ i ∈ {1}  
 __anon__[i] ∀ i ∈ {1,2}
 __anon__[i] ∀ i ∈ {1,2}
 __anon__[i] ∀ i ∈ {1}  

In [58]:
neurons

4-element Array{Array{JuMP.Variable,1},1}:
 #undef
 #undef
 #undef
 #undef

In [56]:
problem.network.layers

3-element Array{Layer,1}:
 Layer([1.0; 1.0], [1.5, 1.5], ReLU())        
 Layer([2.0 2.0; 2.0 2.0], [2.5, 2.5], ReLU())
 Layer([3.0 3.0], [3.5], ReLU())              

In [47]:
problem.network

Network(Layer[Layer([1.0; 1.0], [1.5, 1.5], ReLU()), Layer([2.0 2.0; 2.0 2.0], [2.5, 2.5], ReLU()), Layer([3.0 3.0], [3.5], ReLU())])

In [49]:
neurons

4-element Array{Array{JuMP.Variable,1},1}:
 #undef
 #undef
 #undef
 #undef

In [54]:
all_layers_n  = [length(l.bias) for l in layers]

3-element Array{Int64,1}:
 2
 2
 1

In [44]:
? @variable()

No documentation found.

`JuMP.@variable` is a macro.

```
# 1 method for macro "@variable":
@variable(args...) in JuMP at /Users/christopherlazarus/.julia/v0.6/JuMP/src/macros.jl:871
```


In [38]:
neurons

4-element Array{Array{JuMP.Variable,1},1}:
 __anon__[i] ∀ i ∈ {1}  
 __anon__[i] ∀ i ∈ {1,2}
 __anon__[i] ∀ i ∈ {1,2}
 __anon__[i] ∀ i ∈ {1}  

In [40]:
deltas

4-element Array{Array{JuMP.Variable,1},1}:
 __anon__[i] ∈ {0,1} ∀ i ∈ {1}  
 __anon__[i] ∈ {0,1} ∀ i ∈ {1,2}
 __anon__[i] ∈ {0,1} ∀ i ∈ {1,2}
 __anon__[i] ∈ {0,1} ∀ i ∈ {1}  